In [1]:
%load_ext rpy2.ipython

D:\need_softwares\anaconda\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
%%R

library(MSstats)

setwd('D:/dataset/R downstream analysis/pquant/data/PXD010981-benchmark')

In [5]:
%%R

fileData <- read.csv('new_out_msstats.csv')
DDA2009.proposed <- MSstats::dataProcess(raw = fileData,
                                         logTrans = 2,
                                         normalization = "equalizeMedians",
                                         summaryMethod = "TMP",
                                         maxQuantileforCensored = 0.999,
                                         censoredInt = "NA",
                                         MBimpute = TRUE,
                                         use_log_file = FALSE)

len <- length(levels(DDA2009.proposed$FeatureLevelData$GROUP))
                
tmp <- t(combn(len,2))
matrix_len = length(t(combn(len,2))) / 2

ourMatrix <- matrix(c(0:0),nrow=matrix_len,ncol=len)

for(i in 1:matrix_len){
  ourMatrix[i, tmp[i]] = -1
  ourMatrix[i, tmp[i + matrix_len]] = 1
}

ourCondition <- levels(DDA2009.proposed$ProteinLevelData$GROUP)
tmp_name <- matrix(ourCondition, nr=len, nc=1)
name <- matrix(nr=matrix_len, nc=1)
for(i in 1:matrix_len){
  name[i,1] <- sprintf('%s-%s', tmp_name[tmp[i+matrix_len]], tmp_name[tmp[i]])
}
row.names(ourMatrix) <- name

colnames(ourMatrix) <- ourCondition
DDA2009.comparisons <- groupComparison(contrast.matrix = ourMatrix,
                                       data = DDA2009.proposed,
                                       use_log_file = FALSE)

write.csv(DDA2009.comparisons$ComparisonResult, file="MSstats_output.csv")

INFO  [2021-11-26 16:58:36] ** Features with one or two measurements across runs are removed.
INFO  [2021-11-26 16:58:36] ** Fractionation handled.
INFO  [2021-11-26 16:58:36] ** Updated quantification data to make balanced design. Missing values are marked by NA
INFO  [2021-11-26 16:58:38] ** Log2 intensities under cutoff = 17.611  were considered as censored missing values.
INFO  [2021-11-26 16:58:38] ** Log2 intensities = NA were considered as censored missing values.
INFO  [2021-11-26 16:58:38] ** Use all features that the dataset originally has.
INFO  [2021-11-26 16:58:38] 
 # proteins: 3354
 # peptides per protein: 1-174
 # features per peptide: 1-1
INFO  [2021-11-26 16:58:38] Some proteins have only one feature: 
 sp|A5Z2X5|YP010_YEAST,
 sp|O13516|RS9A_YEAST,
 sp|O13525|COQ4_YEAST,
 sp|O13539|THP2_YEAST,
 sp|P00044|CYC1_YEAST ...
INFO  [2021-11-26 16:58:38] 
                    sample1 sample2 sample3 sample4
             # runs       3       3       3       3
    # bioreplicate

### get MSstats_output.csv
## ------------------------------
### get Proteus_output.csv

In [6]:
import re
import pandas as pd
import numpy as np
from collections import Counter
import os
from pyteomics import mztab


def function(sequence, reference, evi_protein_group):
    evi_sequence = []  # Total cleaning data
    evi_modified_sequence = []
    out_C = []
    evi_modifications = []
    evi_experiment = []

    for i in range(0, len(csv_PeptideSequence)):
        top = csv_PeptideSequence[i]
        while "(" in top:
            a = top.find('(')
            b = top.find(')')
            top = top.replace(top[a:b + 1], '')  # Replace the data in brackets with empty
        result = re.sub('[\W_\d]+', '', top)  # Filter character
        evi_sequence.append(result)
        evi_experiment.append(reference[i][:-5])
    #print(len(evi_sequence))
    #print(len(evi_experiment))

    for i in range(0, len(csv_PeptideSequence)):
        top = csv_PeptideSequence[i]
        k = csv_PeptideSequence[i]
        date = "_"
        #row = []
        #col = []
        next = 0
        while "(" in top:
            a = top.find('(')
            b = top.find(')')
            c = a + next
            d = b + next
            temp = k[c + 1:c + 3]  # Lowercase the first two strings
            temp = temp.lower()
            k = k.replace(k[c + 1:d], temp)
            top = top.replace(top[a:b + 1], '')
            next = 4
        k = k.replace(".", '')
        evi_modified_sequence.append(date + k + date)

    
    for i in range(0, len(csv_PeptideSequence)):
        top = csv_PeptideSequence[i]
        flag = True   # True means this line is unmodified
        out_C = {}
        while "(" in top:
            flag = False
            a = top.find('(')
            b = top.find(')')
            temp = top[a + 1: b]  # Storage modification
            lent = len(top.split(temp)) - 1  # Modification occurrences
            top = top.replace(top[a:b + 1], '')
            out_C[temp] = lent
        
        # Convert dictionary to array
        tem = []
        for key, value in out_C.items():
            if value == 1:  # If it only appears once, no need to display the number of occurrences
                tem.append(key)
            else:
                tem.append(str(value) + ' ' + key)
        tem = ','.join(tem)

        if flag:
            evi_modifications.append("Unmodified")
        else:
            evi_modifications.append(tem)

    evi_protein = []
    for i in range(0, len(evi_protein_group)):
        protein_group = evi_protein_group[i]
        if ";" in protein_group:
            semicolon = protein_group.index(';')
            evi_protein.append(protein_group[:semicolon])
        else:
            evi_protein.append(protein_group)

    return evi_sequence, evi_modified_sequence, evi_modifications, evi_experiment, evi_protein


def get_mztab(pri_mztab, data_dir):
    #df_pri = pd.read_csv(pri_mztab)
    out_mzTab_path = pri_mztab
    mztab_data = mztab.MzTab(out_mzTab_path)
    pep_table = mztab_data.peptide_table
    df_pri = pep_table

    df_pri.to_csv(data_dir + '/pep.csv', index=False)

    return ''





if __name__ == "__main__":
    
    # TODO These code implements data processing
    #data_dir = os.getcwd()
    data_dir = r'D:\dataset\R downstream analysis\pquant\data\PXD010981-benchmark'

    csv = data_dir  + "\\out_msstats.csv"
    #pri_mztab = data_dir  + "/out_mzTab.csv"
    pri_mztab = data_dir  + "\\out.mzTab"

    get_mztab(pri_mztab, data_dir)

    mztab = data_dir + "/pep.csv"
    
    df_csv = pd.read_csv(csv)
    df_mztab = pd.read_csv(mztab)

    csv_PeptideSequence = df_csv['PeptideSequence']
    csv_ProteinName = df_csv['ProteinName']
    csv_experiment = df_csv['Reference']
    csv_PrecursorCharge = df_csv['PrecursorCharge']
    csv_intensity = df_csv['Intensity']

    evi_protein_group = csv_ProteinName
    evi_charge = csv_PrecursorCharge
    evi_intensity = csv_intensity


    evi_sequence, evi_modified_sequence, evi_modifications, evi_experiment, evi_protein = function(csv_PeptideSequence, csv_experiment, evi_protein_group)
    evidence = pd.DataFrame({
        "PeptideSequence": evi_sequence,
        "modified_sequence": evi_modified_sequence,
        "modifications": evi_modifications,
        "protein_group": evi_protein_group,
        "protein": evi_protein,
        "experiment": evi_experiment,
        "charge": evi_charge,
        #"reverse": [],
        #"contaminant": [],
        "intensity": evi_intensity})
    evidence.to_csv(data_dir + "/result_1.csv")
    
    # TODO：The following code implements the VLOOKUP function operation
    data_text = data_dir + "/result_1.csv"
    pep_text = data_dir + "/pep.csv"
    pep = pd.read_csv(pep_text)
    df = pd.read_csv(data_text)
    data = df["PeptideSequence"]
    Pep = pep[["sequence", "accession"]]
    Pep = Pep.drop_duplicates(subset="sequence")  # De-duplicate the second table
    
    df_merge = pd.merge(left=df, right=Pep, left_on="PeptideSequence", right_on="sequence", how='left', )


    if df_merge.columns[0] != 'PeptideSequence':
        tmp = df_merge.iloc[:,1:len(df_merge.columns)]  # Deletes additional data in the first column that is not known when it was generated
        df_merge = tmp
        #print(df_merge)

    df_merge.to_csv(data_dir + "/out_proteus.csv", index=False)

In [8]:
%%R

library(proteus)
setwd('D:/dataset/R downstream analysis/pquant/data/PXD010981-benchmark')
evi <- read.csv("./out_proteus.csv")
meta <- read.csv("./metadata.csv")  # manually annotate
pepdat <- makePeptideTable(evi, meta)
prodat <- makeProteinTable(pepdat)
prodat.med <- normalizeData(prodat)

# PX from the meta's condition, and you can only compare two conditions once.
res <- limmaDE(prodat.med, conditions=c("sample2","sample1"), transform.fun=log2)
write.csv(res, file="Proteus_output2-1.csv")

res <- limmaDE(prodat.med, conditions=c("sample3","sample1"), transform.fun=log2)
write.csv(res, file="Proteus_output3-1.csv")

res <- limmaDE(prodat.med, conditions=c("sample4","sample1"), transform.fun=log2)
write.csv(res, file="Proteus_output3-1.csv")

res <- limmaDE(prodat.med, conditions=c("sample3","sample2"), transform.fun=log2)
write.csv(res, file="Proteus_output3-2.csv")

res <- limmaDE(prodat.med, conditions=c("sample4","sample2"), transform.fun=log2)
write.csv(res, file="Proteus_output4-2.csv")

res <- limmaDE(prodat.med, conditions=c("sample4","sample3"), transform.fun=log2)
write.csv(res, file="Proteus_output4-3.csv")

Then I use Excel to open each comparison file, add [Label] column, finally integrate them into Proteus_output.csv